In [1]:
from core import Seed, init

init()

In [2]:
# load all the companies from Airtable
import airtable

companies = airtable.load_into_pandas()
companies.head()

,id,createdTime,fields.extra_json,fields.Name,fields.Website,fields.Status,fields.Created,fields.Last Modified,fields.Created By.id,fields.Created By.email,fields.Created By.name,fields.Last Modified By.id,fields.Last Modified By.email,fields.Last Modified By.name,fields.Key Product Name,fields.Keywords,fields.Refresh Days,domain
3,rec5nPLxKw5vzPAgP,2024-10-02T16:56:55.000Z,"{""name"":""TBD"",""description"":""TBD is Block\u201...",TBD,https://tbd.website/,Approved,2024-10-02T16:56:55.000Z,2024-10-02T20:34:40.000Z,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,NaN,tbd.website
5,rec7STu2pyWvo9cIp,2024-10-02T16:56:58.000Z,"{""name"":""Veritone"",""description"":""Veritone has...",Veritone,http://www.veritone.com/,Approved,2024-10-02T16:56:58.000Z,2024-10-07T21:34:58.000Z,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,NaN,veritone.com
6,rec7Xm4DXzYQ9uoeb,2024-10-02T16:56:56.000Z,"{""name"":""DoubleDown Interactive LLC"",""descript...",DoubleDown Interactive,https://www.doubledowninteractive.com/jobs/,Approved,2024-10-02T16:56:56.000Z,2024-10-04T19:39:49.000Z,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,NaN,doubledowninteractive.com
7,rec7zrOPWtNbHBjUp,2024-10-08T18:11:30.000Z,NaN,Sourcegraph,https://sourcegraph.com/,Approved,2024-10-08T18:11:30.000Z,2024-10-10T15:02:54.000Z,usrPAGESHARE00000,anonymous+formpage@noreply.airtable.com,Anonymous,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,NaN,sourcegraph.com
8,rec8VeVqBxJsUpNaD,2024-10-02T16:56:56.000Z,"{""name"":""ThousandEyes (part of Cisco)"",""descri...",ThousandEyes,http://www.thousandeyes.com,Approved,2024-10-02T16:56:56.000Z,2024-10-08T18:31:51.000Z,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,usrlUO0c7FWWC9lIZ,keith.trnka@gmail.com,Keith Trnka,NaN,NaN,NaN,thousandeyes.com


In [3]:
sorted_companies = companies.sort_values(by="fields.Last Modified", ascending=False)
sorted_companies[["fields.Name", "fields.Last Modified"]].head(10)

,fields.Name,fields.Last Modified
98,Gable,2024-11-13T17:28:38.000Z
48,Taskrabbit,2024-11-08T18:13:37.000Z
69,Current,2024-10-21T16:19:00.000Z
76,Logic20/20,2024-10-18T21:58:03.000Z
17,Nabla,2024-10-16T17:19:26.000Z
99,Abridge,2024-10-15T17:37:40.000Z
84,Optimize Health,2024-10-14T19:21:57.000Z
34,The Coalition,2024-10-11T16:03:02.000Z
7,Sourcegraph,2024-10-10T15:02:54.000Z
50,Roam,2024-10-10T15:02:51.000Z


In [4]:
from datetime import datetime, timedelta
import os
from typing import Optional

def get_file_age(file_path: str) -> Optional[timedelta]:
    if os.path.exists(file_path):
        file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
        return datetime.now() - file_mod_time
    else:
        return None

# NOTE: It does exact matching on the company name
force_rebuild_companies = {
    # "Sudowrite"
}

def should_rebuild(target: Seed, file_path: str, max_age: timedelta = timedelta(days=7)) -> bool:
    if target.company in force_rebuild_companies:
        return True
    
    age = get_file_age(file_path)
    if age is None:
        return True
    return age > max_age


In [5]:
from loguru import logger

import unified

data_folder = "../output/data"

for _, row in companies.sort_values("fields.Name").iterrows():
    target = airtable.row_to_seed(row)
    output_json = f"{data_folder}/{target.as_path_v2()}.json"

    # NOTE: Without the dropna, it returns NaN which then causes the timedelta to fail
    refresh_days = row.dropna().get("fields.Refresh Days", 21)

    if should_rebuild(target, output_json, max_age=timedelta(days=refresh_days)):
        logger.info(f"Building {output_json}...")

        try:
            unified_result = await unified.run(
                target, 
                # TODO: Allow some customization of these parameters
                num_reddit_threads=10, 
                max_glassdoor_review_pages=5, 
                max_glassdoor_job_pages=0,
                max_news_articles=20,
                )
            
            with open(output_json, 'w') as json_file:
                json_file.write(unified_result.model_dump_json(indent=2))
        except IndexError as e:
            logger.error(f"Error, skipping {target.company}: {e}")



2024-11-13 09:30:56.995 | INFO     | __main__:<module>:15 - Building ../output/data/Current.json...
2024-11-13 09:30:57.432 | INFO     | data_sources.crunchbase.scrapfly_scraper:scrape_company:65 - scraping company: https://www.crunchbase.com/organization/current-powered-by-ge
2024-11-13 09:31:14.821 | INFO     | data_sources.company_webpage:run:60 - search took 1.2 seconds
2024-11-13 09:32:39.239 | INFO     | data_sources.company_webpage:run:63 - scrape took 84.4 seconds
2024-11-13 09:32:48.723 | INFO     | data_sources.company_webpage:run:67 - parse took 9.5 seconds
2024-11-13 09:32:59.875 | INFO     | data_sources.company_webpage:run:73 - summarize took 11.2 seconds
2024-11-13 09:32:59.877 | INFO     | data_sources.company_webpage:run:84 - 37,165 -> 3,944 chars (11%) 
2024-11-13 09:32:59.880 | INFO     | data_sources.company_webpage:run:84 - Extractive fraction: 4% ❌
2024-11-13 09:32:59.881 | INFO     | data_sources.company_webpage:run:84 - Percent of URLs in sources: 100% ✅
2024-11